In [1]:
# geometry description (including boundary/mat names)
from ngsolve import *
from netgen.occ import *
from ngsolve.webgui import *
from ngsolve.webgui import Draw

wp = WorkPlane()
wp.MoveTo(-12,-5).LineTo(-3,-5).NameVertex("fillet")
wp.LineTo(-3,-1).NameVertex("fillet")
wp.LineTo(0,-1).LineTo(0,0).LineTo(-12,0).Close()
geo = wp.Face()
geo = geo.MakeFillet(list(set(geo.vertices["fillet"])),2)
geo = geo + geo.Mirror(Axis((0,0,0),X)).Reversed()
geo = Glue([geo,geo.Mirror(Axis((0,0,0),Y)).Reversed()])
geo.faces.Min(X).name="upperlevel"
geo.faces.Max(X).name="lowerlevel"
geo.edges.name = "wall"
geo.edges.Nearest((0,0)).name = "dam"
#Draw(geo)
geo = OCCGeometry(geo,dim=2)
mesh = Mesh(geo.GenerateMesh(maxh=2))
mesh.Curve(3)
Draw(mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [3]:
order = 2
fes = L2(mesh,order=order)**3
#fes = L2(mesh,order=order,dim=3)

In [4]:
U,V = fes.TnT() # "Trial" and "Test" function
h, hu, hv = U

# initial conditions
h0mat = {"upperlevel" : 10, "lowerlevel" : 2}
U0 = CF((mesh.MaterialCF(h0mat),0,0))

# boundary conditions
hbndreg = mesh.BoundaryCF({"wall" : h, "dam" : 0})
hubndreg = mesh.BoundaryCF({"wall" : -hu, "dam" : 0})
hvbndreg = mesh.BoundaryCF({"wall" : -hv, "dam" : 0})

Ubnd = CF((hbndreg,hubndreg,hvbndreg))

# constant for gravitational force
g=1

In [5]:
def F(U):
    h, hvx, hvy = U
    vx = hvx/h
    vy = hvy/h
    return CF(((hvx,hvy),
               (hvx*vx + 0.5*g*h**2, hvx*vy),
               (hvx*vy, hvy*vy + 0.5*g*h**2)),dims=(3,2))

In [6]:
n = specialcf.normal(mesh.dim)
def Max(u,v):
    return IfPos(u-v,u,v)
def Fmax(A,B): # max. characteristic speed:
    ha, hua, hva = A
    hb, hub, hvb = B
    vnorma = sqrt(hua**2+hva**2)/ha
    vnormb = sqrt(hub**2+hvb**2)/hb
    return Max(vnorma+sqrt(g*A[0]),vnormb+sqrt(g*B[0]))

def Fhatn(U): # numerical flux
    Uhat = U.Other(bnd=Ubnd)
    return (0.5*F(U)+0.5*F(Uhat))*n + Fmax(U,Uhat)/2*(U-Uhat)

In [7]:
def DGBilinearForm(fes,F,Fhatn,Ubnd):
    a = BilinearForm(fes, nonassemble=True)
    a += - InnerProduct(F(U),Grad(V)) * dx
    a += InnerProduct(Fhatn(U),V) * dx(element_boundary=True)
    return a

a = DGBilinearForm(fes,F,Fhatn,Ubnd)

In [13]:
from DGdiffusion import AddArtificialDiffusion

artvisc = Parameter(1.0)
if order > 0:
    AddArtificialDiffusion(a,Ubnd,artvisc,compile=True)

In [14]:
gfu = GridFunction(fes)
gfh, gfhu, gfhv = gfu
gfvu = gfhu/gfh
gfvv = gfhv/gfh
momentum = CF((gfhu,gfhv))
velocity = CF((gfvu,gfvv))
gfu.Set(U0)
scenes = [ \
    Draw(momentum,mesh,"mom"),
    Draw(velocity,mesh,"vel"),
    Draw(gfh,mesh,"h") ]

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

In [15]:
def TimeLoop(a,gfu,dt,T,nsamplings=100,scenes=scenes,multidim_draw=False,md_nsamplings=20):
    if multidim_draw:
        gfu_t = GridFunction(gfu.space,multidim=True)
    #gfu.Set(U0)
    res = gfu.vec.CreateVector()
    fes = a.space
    t = 0; i = 0
    nsteps = int(ceil(T/dt))
    invma = fes.Mass(1).Inverse() @ a.mat
    if multidim_draw:
        gfu_t.AddMultiDimComponent(gfu.vec)
    with TaskManager():
        while t <= T - 0.5*dt:
            res = invma * gfu.vec
            gfu.vec.data -= dt * res
            t += dt
            if (i+1) % int(nsteps/nsamplings) == 0:
                for s in scenes: s.Redraw()
            if multidim_draw and (i+1) % int(nsteps/md_nsamplings) == 0:
                gfu_t.AddMultiDimComponent(gfu.vec)
            i+=1
            print("\rt = {:.10}".format(t),end="")
    for s in scenes: s.Redraw()
    if multidim_draw:
        return gfu_t

In [16]:
gfu.Set(U0)
with TaskManager():
    gfu_t = TimeLoop(a,gfu,dt=0.0025,T=15,multidim_draw=True,md_nsamplings=5)

t = 15.0975

In [17]:
Draw(gfu_t.components[0],mesh,"h",interpolate_multidim=True,animate=True,
     deformation=True, settings = {"camera" :
                                   {"transformations" :
                                    [{ "type": "rotateX", "angle": -20},
                                     { "type": "rotateZ", "angle": 0}]}},
     min=2, max=5, autoscale=False)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {'camera': {'transformations':…

BaseWebGuiScene